In [1]:
import pyfits as pf
import pylab as plt
from scipy import optimize
from scipy.signal import medfilt, find_peaks_cwt
from scipy.ndimage.filters import minimum_filter, maximum_filter, median_filter, convolve
from scipy.ndimage.measurements import label
import numpy as np

In [ ]:
# cd /Users/Carlos/Documents/HERMES/reductions/myherpy/HD1581/

In [ ]:
#Opens a file and subtracts bias from overscann
#In: filename
#out: thisData (data array)
def openFile(fileName):
    thisFile = pf.open(fileName)

    print thisFile[0].header['OBJECT']
    
    gain0_2000  = thisFile[0].header['RO_GAIN']
    gain2000_4000  = thisFile[0].header['RO_GAIN1']

    thisData = thisFile[0].data

    bias0_2000 = np.median(thisData[3:2052,4099:-3])
    bias2000_4000 = np.median(thisData[2059:-3,4099:-3])

    thisData = thisData[:,:4095]

    thisData[:2055] -= bias0_2000
    thisData[2055:] -= bias2000_4000
    
    return thisData



In [ ]:
#Cleans and flattens the flat
#in: raw flat
#out: flattened flat
def make_flat_flat(flat):
    #Flat fielding
    flat_mf = medfilt(flat, [3,9])
    flat_1d = np.sum(flat_mf,axis =0)
    flat_per = np.percentile(flat_1d, 90)
    flat_1d_norm = flat_1d/flat_per
    flat_flat = flat_mf / flat_1d_norm[None,:]
    return flat_flat

In [ ]:
#Convert flat_flat to binary for tracing
def make_flat_flat_bin(flat_flat):
    flat_flat_bin = flat_flat.copy()

    for i in range(flat_flat.shape[1]):
        singleCol = flat_flat[:,i].copy()

        singleMinEnv = convolve(minimum_filter(singleCol,15),[.2,.2,.2,.2,.2])
        singleMin = singleCol - singleMinEnv

        singleMax = convolve(maximum_filter(singleMin,15),[.2,.2,.2,.2,.2])

        fixer = convolve(singleMax, np.ones(200)/200)
        singleMax[singleMax<fixer*.5] = fixer[singleMax<fixer*.5]*.5
        singleColFlat = singleMin/singleMax

        singleColFlat[singleColFlat>.3] = 1
        singleColFlat[singleColFlat<.3] = 0

        flat_flat_bin[:,i] = singleColFlat
    return flat_flat_bin

In [ ]:
def make_fibre_centroids(flat_flat_bin):
    out_array, fibres = label(flat_flat_bin, np.ones((3,3)))
    # print n,'fibres'
    # n-=2 # fibres 252 and 253 are not good for HD1581 epoch 0 

    #create centroid array
    cols = out_array.shape[1]
    fibre_centroids = np.ones((fibres,cols))*np.nan
    for fibre in range(1,fibres+1):
        wRows, wCols = np.where(out_array==fibre)
        print fibre,
        for col in range(max(wCols)+1):
            fibre_centroids[fibre-1, col] = np.average(wRows[wCols==col])
    return fibre_centroids

In [ ]:
#create polynomials from centroids
def make_fibrePolys(fibre_centroids):
    fibrePolys = np.ones((fibre_centroids.shape[0],6))*np.nan
    for y,fibre in enumerate(fibre_centroids):
        fibrePolys[y-1,:] = np.polyfit(range(fibre.shape[0]),fibre,5)
        if np.sum(np.isnan(fibrePolys[y-1,:]))>0:
            print 'Found nan in fibre number',y
            print 'Fibre values',fibre[np.isnan(fibre)]
            print
    return fibrePolys

In [ ]:
#create tramlines from polynomials
def make_tramlines(fibre_centroids, fibrePolys):
    tramlines = (np.ones(fibre_centroids.shape)*np.nan)[:-1]
    thisRange = np.arange(fibre_centroids.shape[1])
    for i,thisPoly in enumerate(fibrePolys[1:]):
        tramlines[i] = fithOrder(thisPoly,thisRange)
    return tramlines

In [ ]:
def fithOrder(thisPoly, thisRange):
    result = thisPoly[0]*thisRange**5
    result += thisPoly[1]*thisRange**4
    result += thisPoly[2]*thisRange**3
    result += thisPoly[3]*thisRange**2
    result += thisPoly[4]*thisRange**1
    result += thisPoly[5]*thisRange**0
    
    return result

In [ ]:
def find_vertical_shift(flat, arc):
    CCTotal = 0
    for column in range(flat.shape[1]):
        thisFlatCol = flat[:,column]
        thisArcCol = arc[:,column]
        CCCurve = np.correlate(thisFlatCol, thisArcCol, mode='full')
        CCTotal += CCCurve

    y = CCTotal[int(CCTotal.shape[0]/2.)+1-5:int(CCTotal.shape[0]/2.)+1+4]
    y /=np.max(y)
    x = np.arange(-4,5)
    x_dense = np.linspace(-4,4)
    p,_ = fit_gaussian([1,3.],y,x )
    shift = p[0]
    return shift
################
##Need to SUBTRACT the result of the gaussian fit 
###to make the 1st curve be like the second (i.e the traces be like the arc)

In [ ]:
def sum_extract(fibre, tramlines, image, numPx):
    
    flux = np.ones(tramlines.shape[1])*np.nan
#     flux1 = np.ones(tramlines.shape[1])*np.nan
#     flux2 = np.ones(tramlines.shape[1])*np.nan
    
    for i,thisCentroid in enumerate(tramlines[fibre]):
#         print thisCentroid
        try:
            fullPx = image[ int(thisCentroid)-numPx : int(thisCentroid)+numPx+1 , i]
            flux[i] = np.sum(fullPx) - fullPx[0]*(thisCentroid%1) - fullPx[-1]*(1-thisCentroid%1)
#         flux1[i] = fullPx[0]*(thisCentroid%1)
#         flux2[i] = fullPx[-1]*(1-thisCentroid%1)
        except:
            print fibre, 'falied'
            print thisCentroid, 'centroid found in index',i
            break
#             print fibre
    return flux

In [ ]:
def extract(tramlines_shifted, data):
    extracted = np.ones(tramlines_shifted.shape)*np.nan
    for fibre in range(tramlines_shifted.shape[0]):
        extracted[fibre] = sum_extract(fibre,tramlines_shifted, data, 4)
    return extracted

In [ ]:
def gaussian(x, mu, sig, ):
    x = np.array(x)
    return np.exp(-np.power(x - mu, 2.) / 2 / np.power(sig, 2.))


def flexi_gaussian(x, mu, sig, power, a, d ):
    x = np.array(x)
    return a* np.exp(-np.power(np.abs((x - mu) * np.sqrt(2*np.log(2))/sig),power))+d

def fit_gaussian(p, flux, x_range):
    a = optimize.leastsq(diff_gaussian, p, args= [flux, x_range])
    return a

def fit_flexi_gaussian(p, flux, x_range):
    a = optimize.leastsq(diff_flexi_gaussian, p, args= [flux, x_range])
    return a

def diff_gaussian(p, args):
    
    flux = args[0]
    x_range = args[1]

    diff = gaussian(x_range, p[0],p[1]) - flux
    return diff

def diff_flexi_gaussian(p, args):
    
    flux = args[0]
    x_range = args[1]
    weights = np.abs(np.gradient(flux)) * (flux+np.max(flux)*.1)
    diff = (flexi_gaussian(x_range, p[0], p[1], p[2], p[3], p[4]) - flux)# *weights
    return diff


In [ ]:
#takes a filename and an array and writes the npy in folder\
def write_NPY(fileName, prefix, postfix, data, folder =""):
    if folder[:-1]!="/": folder += "/"
    outName = folder + prefix + "_" + fileName.split("/")[-1][:-5] + "_" + postfix
    np.save(outName, data)
    print outName, "saved"

In [ ]:
#takes a filename and an array and writes the npy in folder\
def read_NPY(fileName, prefix, postfix, folder =""):
    if folder[:-1]!="/": folder += "/"
    outName = folder + prefix + "_" + fileName.split("/")[-1][:-5] + "_" + postfix
    data = np.load(outName)
    print outName, "read"
    return data